## 第一部分BitFit实战



In [1]:
import os
# 切换到国内镜像站，速度快且稳定
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
from datasets import Dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer
print(1)

1


In [2]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")#要先把数据ds.save_to_disk才能用load_from_disk，现在就是这个情况
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

BloomTokenizerFast(name_or_path='Langboat/bloom-1b4-zh', vocab_size=46145, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [ ]:
def process_func(example):
    Max_LENGTH = 256
    input_ids,attention_mask,labels = [],[],[]
    instruction = tokenizer("\n".join(['Human:'+ example["instruction"],example["input"]]).strip() + "\n\nAssistant:")
    response = tokenizer(example["output"]+tokenizer.eos_token)
    input_ids = instruction["input_ids"] + response["input_ids"]
    attention_mask = instruction["attention_mask"] + response["attention_mask"]
    labels = [-100] * len(instruction["input_ids"])+response["input_ids"]#-100表示不管，只学答案部分
    if len(input_ids)>Max_LENGTH:
        input_ids = input_ids[:Max_LENGTH]
        attention_mask = attention_mask[:Max_LENGTH]
        labels = labels[:Max_LENGTH]
    return{
        "input_ids":input_ids,
        "attention_mask":attention_mask,
        "labels":labels
    }


In [5]:
tokenized_ds = ds.map(process_func,remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/26858 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [6]:
tokenizer.decode(tokenized_ds[1]["input_ids"])


'Human:解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant:4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

Python 3 中，filter() 返回的是一个 迭代器（Iterator），迭代器需要类似于遍历操作比如list去激活才会使用，而tokenizer.decode(token_ids) 函数，它的参数 token_ids 明确要求必须是 序列（Sequence） 类型，比如 List[int]（整数列表）、Tensor 或者 Numpy Array

In [7]:
tokenizer.decode(list(filter(lambda x:x!=-100,tokenized_ds[1]["labels"])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

model.parameters()：返回的是一个个 Tensor（张量/矩阵）。

比如：一个形状为 (3, 4) 的矩阵。param.numel()：意思是 Number of Elements（元素的数量）
对于 (3, 4) 的矩阵，numel() 返回 12。它是在数“箱子里有几个苹果”。
low_cpu_mem_usage=True（低 CPU 内存占用模式）加载模型节省内存


In [11]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh",low_cpu_mem_usage=True)
sum(param.numel() for param in model.parameters())

1303111680

In [12]:
num_param = 0
for name,param in model.named_parameters():
    if "bias" not in name:
        param.requires_grad = False
    else:
        num_param += param.numel()
num_param

544768

In [13]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_eval_batch_size=1,#在验证阶段每张显卡一次读取多少条数据
    gradient_accumulation_steps=8,#每八次更新一次梯度
    logging_steps=10,#每训练多少打印一次日志，主要是loss
    num_train_epochs=1,#整个完整训练集学几遍
    fp16=True,
    gradient_checkpointing=False,
)

Trainer并没有克隆一个新的模型去训练，而是直接修改你传进去的那个 model 对象
每一次 Step（每一个 Batch），model 里的参数（Weights）都在被更新。

如果你在训练途中点了“停止”按钮（Interrupt Kernel），内存里的 model 就会停留在你点击那一瞬间的状态。

In [14]:
trainer = Trainer(
    model = model,
    args = args,
    tokenizer = tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer,padding=True),
)

/var/folders/vt/2rwrwr1j1hn9724wbbsr7jm80000gn/T/ipykernel_3796/3248672971.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
import torch

# 1. 自动选择设备：如果有苹果加速用 mps，否则用 cpu
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [16]:
trainer.train()

/Users/liuzhen/Documents/PEFT Study/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,224616.125000


KeyboardInterrupt: 